[View in Colaboratory](https://colab.research.google.com/github/mohit62/Deep-Learning/blob/master/FacialExpressionClassificationUsingTransferLearning.ipynb)

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
10.327917099
GPU (s):
0.74477314949
GPU speedup over CPU: 13x


#Mount Google Drive to Collab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#Read dataset in form of Pandas Dataframe


In [0]:
import pandas as pd
df=pd.read_csv("drive/My Drive/fer2013.csv")


#Structure Xtrain i.e. rgb_batch and Ytrain i.e. train_pred into proper format

In [0]:
import numpy as np
train_df=df["pixels"].where(df["Usage"]=="Training").dropna()
train_predict=df["emotion"].where(df["Usage"]=="Training").dropna()
train_pred=[]
train_images=[]
for image_no in range(len(train_df)):
  temp=np.reshape(map(float,(train_df[image_no].split(" "))),(48,48))
  train_images.append(temp)
  train_pred.append(train_predict[image_no])
train_images=(np.asarray(train_images)-np.mean(train_images))/np.std(train_images)
rgb_batch = np.repeat(train_images[..., np.newaxis], 3, -1)

train_pred=np.asarray(train_pred)

#Convert Ytrain into Categorical Structure using One Hot encoding

In [0]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_pred)
encoded_Y = encoder.transform(train_pred)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


#Import pretrained VGG16 model without the Fully Connected layer

In [0]:
from keras.applications import VGG16
vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(48, 48, 3))

58900480/58889256 [==============================] - 1s 0us/step


#Network Architecture

In [0]:
from keras.layers import Flatten,Dense,PReLU,Activation
from keras.models import Sequential, Model, load_model
from keras.layers.normalization import BatchNormalization
for layer in vgg_conv.layers[:]:
    layer.trainable = False
   
add_model=Sequential()
add_model.add(Flatten(input_shape=vgg_conv.output_shape[1:],name="flatten"))
# Fully connected layer 1st layer
add_model.add(Dense(256,name="Dense1"))
add_model.add(PReLU())
add_model.add(BatchNormalization())

add_model.add(Dense(256,name="Dense2"))
add_model.add(PReLU())
add_model.add(BatchNormalization())
add_model.add(Dense(7, activation='softmax'))
model = Model(inputs=vgg_conv.input, outputs=add_model(vgg_conv.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

#Model Compilation

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])


#Fit Model

In [0]:
model.fit(rgb_batch, dummy_y,batch_size=128,epochs=5,validation_split=0.1)

Train on 25838 samples, validate on 2871 samples
Epoch 1/5
25838/25838 [==============================] - 18s 706us/step - loss: 0.1789 - acc: 0.9395 - val_loss: 1.9037 - val_acc: 0.6308
Epoch 2/5
25838/25838 [==============================] - 19s 719us/step - loss: 0.1760 - acc: 0.9400 - val_loss: 1.9909 - val_acc: 0.6224
Epoch 3/5
25838/25838 [==============================] - 18s 705us/step - loss: 0.1743 - acc: 0.9414 - val_loss: 2.0607 - val_acc: 0.6297
Epoch 4/5
25838/25838 [==============================] - 18s 706us/step - loss: 0.1738 - acc: 0.9412 - val_loss: 1.9669 - val_acc: 0.6287
Epoch 5/5
25838/25838 [==============================] - 18s 705us/step - loss: 0.1742 - acc: 0.9416 - val_loss: 1.9808 - val_acc: 0.6290


##Structure Xtest i.e. rgb_batch_test and Ytest i.e. test_pred into proper format

In [0]:
test_df=df["pixels"].where(df["Usage"]=="PublicTest").dropna()
test_predict=df["emotion"].where(df["Usage"]=="PublicTest").dropna()
test_pred=[]
test_images=[]
try:
  for image_no in range(28709,len(test_df)+28709):
    temp=np.reshape(map(float,(test_df[image_no].split(" "))),(48,48))
    test_images.append(temp)
    test_pred.append(test_predict[image_no])
except:
  print(image_no)
test_images=(np.asarray(test_images)-np.mean(test_images))/np.std(test_images)
rgb_batch_test = np.repeat(test_images[..., np.newaxis], 3, -1)
test_pred=np.asarray(test_pred)

#Convert Ytest into categorical format using One hot encoding

In [0]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(test_pred)
encoded_Y = encoder.transform(test_pred)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(encoded_Y)


#Evaluate Model on the test data

In [0]:
model.evaluate(rgb_batch_test,dummy_y_test, verbose=1)

3589/3589 [==============================] - 4s 1ms/step


[2.075911043812286, 0.6140986347504066]